## 대화 요약 코드

In [1]:
# 원시 말뭉치 에서 기준에 맞게 말뭉치 추출
# 기준 : 첫번째 form 에 [사설] 이 들어가 있고, 220 어절 이상인 문서
import json
import glob

json_list = glob.glob("../../말뭉치/원시_말뭉치/대화 요약/json/*.json")

for each_json in json_list :
    with open(f"{each_json}") as json_f :
        json_data = json.load(json_f)
        for json_document in json_data["document"] :
            form = ""
            if "[사설]" in json_document["paragraph"][0]["form"] :
                for json_document_paragraph in json_document["paragraph"] :
                    form += json_document_paragraph["form"] + "\n"
                if len(form.split(" ")) >= 220 :
                    switch_json = {}
                    switch_json["id"] = json_document["id"]
                    switch_json["metadata"] = json_document["metadata"]
                    switch_json["paragraph"] = json_document["paragraph"]
                    with open(f"../../말뭉치/정제_말뭉치/대화 요약/json/{json_document['id']}.json", "w") as json_file :
                        json.dump(switch_json, json_file, ensure_ascii=False, indent=4)
                        json_file.close()
                    with open(f"../../말뭉치/정제_말뭉치/대화 요약/txt/{json_document['id']}.txt", "w") as txt_file :
                        txt_file.write(form)
                        txt_file.close()


In [28]:
# 문서로 나눈 말뭉치를 작업자가 작업 하는 문서로 변경 하는 작업
# 작업 하는 문서 -> [사설] 없애고, ". "으로 분리해서 ".\n" 을 추가 해서 line by line 으로 분리

from glob import glob

txt_file_list = glob("../../말뭉치/정제_말뭉치/대화 요약/txt_4731/*.txt")

for txt_file in txt_file_list :
    txt_name = txt_file.split("/")[6]
    txt_number = txt_name.split(".")[1]
    form = []
    form_index = 0
    with open(txt_file, "r") as f:
        for line in f :
            for form_line in line.split(". ") :
                form_index += 1
                if "[사설]" in form_line :
                    form.append(f"{txt_number}.{form_index} : " + form_line.split("[사설]")[1].strip())
                else :
                    if "\n" in form_line :
                        form.append(f"{txt_number}.{form_index} : " + form_line[0:len(form_line)-1])
                    else :
                        form.append(f"{txt_number}.{form_index} : " + form_line + ".")
    with open(f"../../말뭉치/2차정제_말뭉치/대화 요약/txt/{txt_name}", "w+") as txt:
        for form_line in form :
            txt.write(form_line + "\n")
        txt.close()

In [43]:
# 1차 라벨링 결과물을 가지고 핵심 문장을 txt에 추가
import jsonlines
from glob import glob
SOURCE_FILE = "./"
WORKER = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]


# 각 jsonl loop
with jsonlines.open("../../말뭉치/라벨링_결과물/20240305/admin.jsonl") as jsonl :

    # jsonl 에서 각 문서를 loop
    for line in jsonl :
        sentence_list = []
        form = []
        new_form = []
        form = line['text'].split("\n")
        form = [v for v in form if v]
        import_sentence = []

        for form_line in form :
            new_form.append(form_line.split(" : ")[1])
        for line_entities in line["entities"] :
            sentence_list.append(line['text'][line_entities['start_offset']:line_entities['end_offset']])        
        for sentence_number in sentence_list :
            for form_line in form :
                if sentence_number in form_line :
                    import_sentence.append(form_line.split(" : ")[1])
        
        new_form.append("\n")
        new_form.append("[핵심 문장]")
        new_form = new_form + import_sentence
        
        with open("../../말뭉치/라벨링_결과물/20240305/admin_txt.txt", "w") as txt_file :
            txt_file.write("\n".join(new_form))
            txt_file.close()
        